In [20]:
import pandas as pd
from nltk.corpus import stopwords
import spacy

# Cargar datos
df = pd.read_csv("diario_emociones.csv")

# Cargar modelo de lenguaje en español
nlp = spacy.load("es_core_news_sm")

# Función para limpiar texto
def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Tokenizar y eliminar stopwords
    doc = nlp(texto)
    tokens = [token.text for token in doc if not token.is_stop and token.is_alpha]
    return " ".join(tokens)

# Aplicar limpieza
df["texto_limpio"] = df["texto"].apply(limpiar_texto)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convertir texto a vectores TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["texto_limpio"]).toarray()

# Convertir emociones a etiquetas numéricas
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["emocion"])

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation="relu"))  # Capa de entrada
model.add(Dropout(0.5))  # Regularización
model.add(Dense(64, activation="relu"))  # Capa oculta
model.add(Dropout(0.5))  # Regularización
model.add(Dense(len(label_encoder.classes_), activation="softmax"))  # Capa de salida

# Compilar el modelo
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Resumen del modelo
model.summary()

C:\Users\walte\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           3,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,781 (46.02 KB)

 Trainable params: 11,781 (46.02 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from sklearn.model_selection import train_test_split

# Dividir datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6562 - loss: 1.1557 - val_accuracy: 1.0000 - val_loss: 0.0201
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9984 - loss: 0.0554 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9994 - loss: 0.0193 - val_accuracy: 1.0000 - val_loss: 4.0185e-04
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9999 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 1.8499e-04
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9996 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 1.3118e-04
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9994 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 8.2303e-05
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9997 - loss: 0.0026 - val_accuracy: 1.0000 - val_loss: 4.7466e-05
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.002

In [24]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida: {loss}, Precisión: {accuracy}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 1.0000 - loss: 2.2027e-05 
Pérdida: 2.6218898710794747e-05, Precisión: 1.0


In [25]:
def predecir_emocion(texto):
    # Limpiar el texto
    texto_limpio = limpiar_texto(texto)
    # Convertir a vector
    texto_vector = vectorizer.transform([texto_limpio]).toarray()
    # Predecir emoción
    prediccion = model.predict(texto_vector)
    emocion = label_encoder.inverse_transform([prediccion.argmax()])[0]
    return emocion

In [26]:
def chatbot_respuesta(texto):
    emocion = predecir_emocion(texto)
    respuestas = {
        "tristeza": "Parece que te sientes triste. ¿Te gustaría probar una meditación guiada?",
        "ansiedad": "Detecté que estás ansioso. ¿Quieres hacer un ejercicio de respiración?",
        "felicidad": "¡Me alegra que te sientas feliz! Sigue así.",
        "enojo": "Parece que estás enojado. ¿Te gustaría hablar sobre lo que te molesta?",
        "estrés": "Parece que estás estresado. ¿Te gustaría escuchar una meditación para relajarte?"
    }
    return respuestas.get(emocion, "No estoy seguro de cómo ayudarte. ¿Puedes contarme más?")

In [27]:
texto_usuario = "Hoy me siento muy estresado por el trabajo."
respuesta = chatbot_respuesta(texto_usuario)
print(respuesta)  # Salida: "Parece que estás estresado. ¿Te gustaría escuchar una meditación para relajarte?"

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Parece que estás enojado. ¿Te gustaría hablar sobre lo que te molesta?


In [18]:
import pandas as pd
import numpy as np
from faker import Faker

fake = Faker('es_ES')
emociones = ["felicidad", "tristeza", "enojo", "ansiedad", "estrés"]
n_registros = 5000 

data = []
for _ in range(n_registros):
    emocion = np.random.choice(emociones)
    if emocion == "felicidad":
        texto = fake.sentence(nb_words=10, ext_word_list=["logro", "familia", "éxito", "risas", "celebración"])
    elif emocion == "tristeza":
        texto = fake.sentence(nb_words=10, ext_word_list=["solitario", "llorar", "vacío", "pérdida", "nostalgia"])
    elif emocion == "enojo":
        texto = fake.sentence(nb_words=10, ext_word_list=["injusticia", "mentira", "traición", "ira", "conflicto"])
    elif emocion == "ansiedad":
        texto = fake.sentence(nb_words=10, ext_word_list=["miedo", "preocupación", "incertidumbre", "nervios", "presión"])
    else:
        texto = fake.sentence(nb_words=10, ext_word_list=["agobio", "fatiga", "presión", "caos", "sobrecarga"])
    
    data.append({
        "fecha": fake.date_between(start_date='-3y', end_date='today').strftime('%Y-%m-%d'),
        "texto": texto.capitalize(),
        "emocion": emocion
    })

df = pd.DataFrame(data)
df.to_csv("diario_emociones.csv", index=False, encoding="utf-8-sig")